In [11]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import re
import nltk
import seaborn as sns
#from gensim.models import word2vec
from tqdm import tqdm
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr
import pandas as pd
import pickle


#pickle_file = open("RQ3_Input_Dataset2",'rb')
##--LDA File
## forTSNE_Data_Cleaned_08092020_083631.df
##forTSNE_Data_Cleaned_07092020_113759.df
##--Sentiment file
##FINAL_Dataset1_Pickle.data 
##FINAL_Dataset2_Pickle.data
##RQ3_Input_Dataset1.data
##RQ3_Input_Dataset2.data

df = pd.read_pickle("RQ3_Input_Dataset2.data")

In [2]:
df.head()

,Org,OrgSector,RverMonthYear,RverReviewer,RverPosition,RverLoc,RverRecom,RvReviewAbout,RvScore,RvComment,...,featureSentiScoreBool,featureSentiScorenormalized,RvScoreScaled,RvScoreWorkAtmosphereScaled,RvScoreCohesionAmongColleaguesScaled,RvScoreEqualRightsScaled,RvScoreDealingWithOlderColleaguesScaled,RvScoreEnvironmentalSocialAwarenessScaled,RvScoreWorkLifeBalanceScaled,Mean_value
1,Instone Real Estate Group AG,Immobilien,May 2020,3,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung im Bereich Adm...,Empfohlen,Overall,4.5,Wenn Transparenz und Wertschätzung aufeinande...,...,1.0,0.345304,0.90,1.0,1.0,4.7,1.0,4.7,0.8,4.700000
2,Instone Real Estate Group AG,Immobilien,October 2019,4,Ex-Angestellte/r oder Arbeiter/in,Hat im Bereich Personal / Aus- und Weiterbildu...,Nicht empfohlen,Overall,3.2,Anspruch und Wirklichkeit klaffen auseinander...,...,1.0,0.028910,0.64,0.6,0.6,0.6,0.8,0.4,0.6,3.028571
3,Instone Real Estate Group AG,Immobilien,October 2019,5,Führungskraft / Management,Hat zum Zeitpunkt der Bewertung bei Instone Re...,Empfohlen,Overall,4.8,Toller Arbeitgeber Tolle Truppe Auch bei lä...,...,1.0,0.012521,0.96,1.0,1.0,1.0,1.0,1.0,0.8,4.828571
4,Instone Real Estate Group AG,Immobilien,September 2019,6,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung im Bereich Bes...,Empfohlen,Overall,4.6,Glücksfall - vom Konzern zum Mittelstand! Ang...,...,1.0,0.314862,0.92,1.0,0.8,0.8,1.0,1.0,1.0,4.657143
5,Instone Real Estate Group AG,Immobilien,August 2019,8,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung bei Instone Re...,Empfohlen,Overall,4.5,100% Zufriedenheit Neue sinnvolle Wege zum Zi...,...,1.0,0.076449,0.90,1.0,1.0,0.8,1.0,0.6,1.0,4.500000


In [12]:
pd.options.display.max_rows

60

In [13]:
df.OrgSector.unique()

array(['Immobilien', 'Elektro/Elektronik', 'Sonstige Branchen',
       'Maschinenbau', 'Handel', 'Internet', 'Medizin/Pharma', 'IT',
       'Automobil', 'Industrie', 'Dienstleistung', 'Energie',
       'Marketing/Werbung/PR', 'Nahrungsmittel/Landwirtschaft', 'Chemie',
       'Finanz', 'Banken', 'Transport/Verkehr/Logistik',
       'Telekommunikation', 'Bau/Architektur', 'Versicherung',
       'Kunst/Kultur', 'Textil', ' Immobilien', 'Medien',
       'Personalwesen & -beschaffung', ' Industrie', 'Sport/Beauty',
       'Druck/Verpackung', 'Medizintechnik', 'Forschung/Entwicklung',
       'Beratung/Consulting'], dtype=object)

In [14]:
conditions = [(df["OrgSector"] == 'Immobilien'),
              (df["OrgSector"] == 'Elektro/Elektronik'),
              (df["OrgSector"] == 'Sonstige Branchen'),
              (df["OrgSector"] == 'Maschinenbau'),
              (df["OrgSector"] == 'Handel'),
              (df["OrgSector"] == 'Internet'),
              (df["OrgSector"] == 'Medizin/Pharma'),
              (df["OrgSector"] == 'IT'),
              (df["OrgSector"] == 'Automobil'),
              (df["OrgSector"] == 'Industrie'),
              (df["OrgSector"] == 'Dienstleistung'),
              (df["OrgSector"] == 'Energie'),
              (df["OrgSector"] == 'Marketing/Werbung/PR'),
              (df["OrgSector"] == 'Nahrungsmittel/Landwirtschaft'),              
              (df["OrgSector"] == 'Chemie'),
              (df["OrgSector"] == 'Finanz'),
              (df["OrgSector"] == 'Banken'),
              (df["OrgSector"] == 'Transport/Verkehr/Logistik'),
              (df["OrgSector"] == 'Telekommunikation'),
              (df["OrgSector"] == 'Bau/Architektur'),
              (df["OrgSector"] == 'Versicherung'),
              (df["OrgSector"] == 'Kunst/Kultur'),
              (df["OrgSector"] == 'Textil'),
              (df["OrgSector"] == ' Immobilien'),
              (df["OrgSector"] == 'Medien'),
              (df["OrgSector"] == 'Personalwesen & -beschaffung'),
              (df["OrgSector"] == ' Industrie'),
              (df["OrgSector"] == 'Sport/Beauty'),
              (df["OrgSector"] == 'Druck/Verpackung'),
              (df["OrgSector"] == 'Medizintechnik'),
              (df["OrgSector"] == 'Forschung/Entwicklung'),
              (df["OrgSector"] == 'Beratung/Consulting')
              ]
choices_NewOrgSector = ['Finance, Insurance, Real Estate', 'Manufacturing', 'Others', 'Manufacturing', 'Retail Trade',
                        'Services','Manufacturing','Manufacturing','Manufacturing','Manufacturing','Services',
                        'Transportation & Public Utilities','Services','Agriculture, Forestry, Fishing','Manufacturing',
                        'Finance, Insurance, Real Estate','Finance, Insurance, Real Estate','Transportation & Public Utilities',
                        'Transportation & Public Utilities','Construction','Finance, Insurance, Real Estate','Services',
                        'Manufacturing','Finance, Insurance, Real Estate','Services','Services','Manufacturing',
                        'Services','Manufacturing','Manufacturing','Services','Services']
df["NewOrgSector"] = np.select(conditions, choices_NewOrgSector, default='NaN')

In [15]:
df.head()

,Org,OrgSector,RverMonthYear,RverReviewer,RverPosition,RverLoc,RverRecom,RvReviewAbout,RvScore,RvComment,...,featureSentiScorenormalized,RvScoreScaled,RvScoreWorkAtmosphereScaled,RvScoreCohesionAmongColleaguesScaled,RvScoreEqualRightsScaled,RvScoreDealingWithOlderColleaguesScaled,RvScoreEnvironmentalSocialAwarenessScaled,RvScoreWorkLifeBalanceScaled,Mean_value,NewOrgSector
1,Instone Real Estate Group AG,Immobilien,May 2020,3,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung im Bereich Adm...,Empfohlen,Overall,4.5,Wenn Transparenz und Wertschätzung aufeinande...,...,0.345304,0.90,1.0,1.0,4.7,1.0,4.7,0.8,4.700000,"Finance, Insurance, Real Estate"
2,Instone Real Estate Group AG,Immobilien,October 2019,4,Ex-Angestellte/r oder Arbeiter/in,Hat im Bereich Personal / Aus- und Weiterbildu...,Nicht empfohlen,Overall,3.2,Anspruch und Wirklichkeit klaffen auseinander...,...,0.028910,0.64,0.6,0.6,0.6,0.8,0.4,0.6,3.028571,"Finance, Insurance, Real Estate"
3,Instone Real Estate Group AG,Immobilien,October 2019,5,Führungskraft / Management,Hat zum Zeitpunkt der Bewertung bei Instone Re...,Empfohlen,Overall,4.8,Toller Arbeitgeber Tolle Truppe Auch bei lä...,...,0.012521,0.96,1.0,1.0,1.0,1.0,1.0,0.8,4.828571,"Finance, Insurance, Real Estate"
4,Instone Real Estate Group AG,Immobilien,September 2019,6,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung im Bereich Bes...,Empfohlen,Overall,4.6,Glücksfall - vom Konzern zum Mittelstand! Ang...,...,0.314862,0.92,1.0,0.8,0.8,1.0,1.0,1.0,4.657143,"Finance, Insurance, Real Estate"
5,Instone Real Estate Group AG,Immobilien,August 2019,8,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung bei Instone Re...,Empfohlen,Overall,4.5,100% Zufriedenheit Neue sinnvolle Wege zum Zi...,...,0.076449,0.90,1.0,1.0,0.8,1.0,0.6,1.0,4.500000,"Finance, Insurance, Real Estate"


In [16]:
df['NewOrgSector'].value_counts()

Manufacturing                        3704
Services                             1019
Transportation & Public Utilities     983
Finance, Insurance, Real Estate       886
Retail Trade                          365
Others                                 51
Construction                           31
Agriculture, Forestry, Fishing         23
Name: NewOrgSector, dtype: int64

In [17]:
df_Manufacturing = df[df['NewOrgSector'] == 'Manufacturing']
df_Transportation = df[df['NewOrgSector'] == 'Transportation & Public Utilities']
df_Services = df[df['NewOrgSector'] == 'Services']
df_Finance = df[df['NewOrgSector'] == 'Finance, Insurance, Real Estate']
df_Retail  = df[df['NewOrgSector'] == 'Retail Trade']
df_Others = df[df['NewOrgSector'] == 'Others']
df_Construction = df[df['NewOrgSector'] == 'Construction']
df_Agriculture = df[df['NewOrgSector'] == 'Agriculture, Forestry, Fishing']

listOfDataframe = [df_Manufacturing, df_Transportation, df_Services, df_Finance, df_Retail, df_Others, df_Construction, df_Agriculture]
#gapminder_2002 = gapminder[gapminder['year']==2002]

In [9]:
def calculate_pValue(df_Manufacturing, df_Construction):
    df_Construction =df_Construction.loc[:,'RvScore']
    #params = np.append(lm.intercept, lm.coef)
    #predictions = lm.predict(train_x_)
    
    dfnewX = pd.DataFrame({"Constant": np.ones(len(df_Manufacturing))}).join(pd.DataFrame(df_Manufacturing))
    MSE = (sum((df_Construction - predictions) ** 2)) / (len(dfnewX) - len(dfnewX.columns))

    # Note if you don't want to use a DataFrame replace the two lines above with
    # newX = np.append(np.ones((len(X),1)), X, axis=1)
    # MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

    #var_b = variance
    print("---------------dfnewX-----------------")
    print(dfnewX)
    var_b = MSE * (np.linalg.inv(np.dot(dfnewX.T, dfnewX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params / sd_b
    print("==(len(dfnewX) - len(dfnewX[0])== :: ", (len(dfnewX) - 1))
    p_values = [2 * (1 - stats.t.cdf(np.abs(i), (len(dfnewX) - 1))) for i in ts_b]


    sd_b = np.round(sd_b, 3)
    ts_b = np.round(ts_b, 3)
    p_values = np.round(p_values, 3)
    params = np.round(params, 4)

    myDF3 = pd.DataFrame()
    myDF3["Coefficients"], myDF3["Standard Errors"], myDF3["tvalues"], myDF3["Probabilities"] = [params, sd_b, ts_b,
                                                                                                  p_values]
    #print(myDF3)
    return myDF3.loc[1:, "Probabilities"], myDF3.loc[1:, "tvalues"]

In [10]:
for i in range(0, 23):
    if i in [12, 20, 22]:
        continue
    col_name = 'topic' + str(i)
    df_Manufacturing['col_name'] = df_Manufacturing[col_name]
    
df_Manufacturing['col_name']

10      0.000000
11      0.000000
12      0.000000
13      0.000000
14      0.010753
          ...   
7045    0.000000
7046    0.000000
7047    0.000000
7048    0.000000
7049    0.018868
Name: col_name, Length: 3704, dtype: float64

In [18]:
pd.set_option('display.max_columns', None)
df_Manufacturing.head()

,Org,OrgSector,RverMonthYear,RverReviewer,RverPosition,RverLoc,RverRecom,RvReviewAbout,RvScore,RvComment,RvScoreWorkAtmosphere,RvScoreCohesionAmongColleagues,RvScoreEqualRights,RvScoreDealingWithOlderColleagues,RvScoreEnvironmentalSocialAwareness,Corona1,Corona2,Corona3,RvScoreWorkLifeBalance,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic20,topic21,topic22,forTSNEYear,forTSNECity,topic0_max,topic1_max,topic2_max,topic3_max,topic4_max,topic5_max,topic6_max,topic7_max,topic8_max,topic9_max,topic10_max,topic11_max,topic13_max,topic14_max,topic15_max,topic16_max,topic17_max,topic18_max,topic19_max,topic21_max,score101,topicXmaxboolXcss0,topicXmaxboolXcss1,topicXmaxboolXcss2,topicXmaxboolXcss3,topicXmaxboolXcss4,topicXmaxboolXcss5,topicXmaxboolXcss6,topicXmaxboolXcss7,topicXmaxboolXcss8,topicXmaxboolXcss9,topicXmaxboolXcss10,topicXmaxboolXcss11,topicXmaxboolXcss13,topicXmaxboolXcss14,topicXmaxboolXcss15,topicXmaxboolXcss16,topicXmaxboolXcss17,topicXmaxboolXcss18,topicXmaxboolXcss19,topicXmaxboolXcss21,NewRverCategory,PreviousVsCurrentFlag,NewRverCategory_Employee,NewRverCategory_Management,NewRverCategory_Qualification,NewRverCategory_TemporaryEmployed,featureYear,state,featureState,featurePosition,featureOrg,featureRecom,featureSentiScoreBool,featureSentiScorenormalized,RvScoreScaled,RvScoreWorkAtmosphereScaled,RvScoreCohesionAmongColleaguesScaled,RvScoreEqualRightsScaled,RvScoreDealingWithOlderColleaguesScaled,RvScoreEnvironmentalSocialAwarenessScaled,RvScoreWorkLifeBalanceScaled,Mean_value,NewOrgSector
10,Isra Vision AG,Elektro/Elektronik,August 2016,3,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung bei ISRA Visio...,Empfohlen,Overall,4.3,interessante Aufgaben in internationalem Umfe...,4.0,5.0,5.0,4.0,3.0,NA,NA,NA,4.0,0.000000,0.000000,0.204355,0.000000,0.000000,0.000000,0.000000,0.000000,0.107279,0.179862,0.000000,0.210268,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.123449,0.000000,0.000000,0.000000,2016,Darmstadt,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.1042,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.02191,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Employee,1,1,0,0,0,0.64,Hessen,0.41,0.07,0.4681,1.0,1.0,0.012286,0.86,0.8,1.0,1.0,0.8,0.6,0.8,4.185714,Manufacturing
11,Isra Vision AG,Elektro/Elektronik,February 2014,3,Führungskraft / Management,Hat zum Zeitpunkt der Bewertung bei ISRA Visio...,,Overall,4.0,Hohe Anforderungen - Einsatz wird aber belohn...,4.0,4.0,5.0,4.0,4.0,NA,NA,NA,4.0,0.000000,0.000000,0.281147,0.000000,0.296545,0.000000,0.000000,0.000000,0.090672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.095640,0.000000,0.000000,0.000000,0.000000,0.089639,0.000000,0.000000,2014,Darmstadt,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.8359,0.0,0.0,0.0,0.0,0.247882,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,Management,1,0,1,0,0,0.45,Hessen,0.41,0.79,0.4681,0.5,1.0,0.098556,0.80,0.8,0.8,1.0,0.8,0.8,0.8,4.142857,Manufacturing
12,Isra Vision AG,Elektro/Elektronik,September 2013,4,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung im Bereich For...,,Overall,4.2,Spannende Aufgaben und Spitzenteams Hier wird...,4.2,4.2,4.2,4.2,4.2,NA,NA,NA,4.2,0.000000,0.000000,0.119252,0.000000,0.193296,0.000000,0.056526,0.000000,0.057513,0.000000,0.176523,0.000000,0.000000,0.000000,0.000000,0.220884,0.097179,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013,Darmstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3.3874,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.000000,0.0,0.748221,0.0,0.0,0.0,0.0,0.0,Employee,1,1,0,0,0,0.36,Hessen,0.41,0.07,0.4681,0.5,1.0,0.399387,0.84,4.2,4.2,4.2,4.2,4.2,4.2,4.200000,Manufacturing
13,Isra Vision AG,Elektro/Elektronik,September 2013,5,Angestellte/r oder Arbeiter/in,Hat zum Zeitpunkt der Bewertung im Bereich For...,,Overall,3.8,"Schon lange dabei, immer neue Herausforderung...",3.8

In [19]:
noOfTopics = 0

for name in df_Manufacturing.loc[:,'topic0':'topic22']:
    if 'topic' in name:
        noOfTopics +=1

In [20]:
#arr = df.to_numpy()
arr_df_Manufacturing_topic0 = df_Manufacturing["topic0"].to_numpy()
arr_df_Manufacturing_topic0

array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.01886822])

In [21]:
#arr = df.to_numpy()

arr_df_Services_topic0 = df_Services["topic0"].to_numpy()
arr_df_Services_topic0

array([0.01204824, 0.01587305, 0.        , ..., 0.        , 0.02325588,
       0.0136987 ])

In [22]:
# Use scipy.stats.ttest_ind_from_stats.
stats.ttest_ind(arr_df_Manufacturing_topic0, arr_df_Services_topic0)


Ttest_indResult(statistic=3.9080338450610648, pvalue=9.435327958224059e-05)

In [16]:
noOfTopics = 0

for name in df_Manufacturing.loc[:,'topic0':'topic22']:
    if 'topic' in name:
        noOfTopics +=1
print("no of topics ::", noOfTopics)

for c in range(noOfTopics):
    if c not in [12,20,22]:
        topic_num = 'topic' + str(c)
        arr_df_Manufacturing_topic0 = df_Manufacturing[topic_num].to_numpy()
        arr_df_Services_topic0 = df_Services[topic_num].to_numpy()
        # Use scipy.stats.ttest_ind_from_stats.
        print(stats.ttest_ind(arr_df_Manufacturing_topic0, arr_df_Services_topic0))
    
    

no of topics :: 23
Ttest_indResult(statistic=3.9080338450610648, pvalue=9.435327958224059e-05)
Ttest_indResult(statistic=-3.8742380549683717, pvalue=0.00010840801821884096)
Ttest_indResult(statistic=2.2326272785732004, pvalue=0.02562024372633525)
Ttest_indResult(statistic=-1.0134568752235515, pvalue=0.31089392106705593)
Ttest_indResult(statistic=0.19434575937147197, pvalue=0.8459135197931118)
Ttest_indResult(statistic=1.6269482995467468, pvalue=0.10381481469650479)
Ttest_indResult(statistic=0.2894111773999139, pvalue=0.772279450971371)
Ttest_indResult(statistic=-0.64318061834991, pvalue=0.5201381461180563)
Ttest_indResult(statistic=2.2195326386367906, pvalue=0.026497852666979783)
Ttest_indResult(statistic=1.3353649595548758, pvalue=0.18182130720059383)
Ttest_indResult(statistic=0.9406976131959954, pvalue=0.3469079717248885)
Ttest_indResult(statistic=-1.0324888772855136, pvalue=0.30189604378457885)
Ttest_indResult(statistic=-0.7847851369498409, pvalue=0.432618933649075)
Ttest_indResult(

In [17]:
noOfTopics = 0
#listOfDataframe = ['df_Manufacturing', 'df_Transportation', 'df_Services', 'df_Finance', 'df_Retail', 'df_Others', 'df_Construction', 'df_Agriculture']
listOfDataframe = ['df_Manufacturing', 'df_Transportation', 'df_Services', 'df_Finance', 'df_Retail', 'df_Others', 'df_Construction', 'df_Agriculture']
for name in df_Manufacturing.loc[:,'topic0':'topic22']:
    if 'topic' in name:
        noOfTopics +=1
print("no of topics ::", noOfTopics)

for c in range(noOfTopics):
    if c not in [12,20,22]:
        topic_num = 'topic' + str(c)
        print("topic number::", topic_num)
        for j in (listOfDataframe):
            print("j value ::")
            count12312 = 0
            for m in listOfDataframe:
                if (m !=j):   
                    arr_df_outerSectore = j[topic_num].to_numpy()
                    arr_df_innerSector = m[topic_num].to_numpy()
                    # Use scipy.stats.ttest_ind_from_stats.
                    print(stats.ttest_ind(arr_df_outerSectore, arr_df_innerSector))
                    count12312=+1
                    
                    if (count12312>3):
                        break
                
                    
    
    

no of topics :: 23
topic number:: topic0
j value ::


TypeError: string indices must be integers

In [23]:
noOfTopics = 0
dfcolumns = ['Manufacturing', 'Transportation', 'Services', 'Finance', 'Retail', 'Others', 'Construction', 'Agriculture']
listOfDataframe = [df_Manufacturing, df_Transportation, df_Services, df_Finance, df_Retail, df_Others, df_Construction, df_Agriculture]
#listOfDataframe = [df_Manufacturing, df_Transportation,df_Services]

for name in df_Manufacturing.loc[:,'topic0':'topic22']:
    if 'topic' in name:
        noOfTopics +=1
print("no of topics ::", noOfTopics)

for c in range(noOfTopics):
    print('====================New CSV==================')
    print(c)
    if c not in [12,20,22]:
        topic_num = 'topic' + str(c)
        #print("topic number::", topic_num)
        dft_testScore = pd.DataFrame(columns = dfcolumns)
        for j in (listOfDataframe):
            #print("j value ::", j)
            #count12312 = 0
            rowStat =[]
            rowPval =[]
            for m in listOfDataframe:
                #print("m value ::", m)
                #print(j[topic_num])
                #if (m == j):   
                arr_df_outerSectore = j[topic_num].to_numpy()
                #print("j head::", j.head(5))
                arr_df_innerSector = m[topic_num].to_numpy()
                # Use scipy.stats.ttest_ind_from_stats.
                t_testScore = stats.ttest_ind(arr_df_outerSectore, arr_df_innerSector)
               # print("statistic value ::==================",t_testScore.statistic)
                rowStat.append(t_testScore.statistic)
                rowPval.append(t_testScore.pvalue)
            a_series = pd.Series(rowStat, index = dft_testScore.columns)
            b_series = pd.Series(rowPval, index = dft_testScore.columns)
            dft_testScore = dft_testScore.append(a_series, ignore_index=True)
            dft_testScore = dft_testScore.append(b_series, ignore_index=True)
        dft_testScore.to_excel("T_attempt3_testScore_topic" + str(c) + ".xlsx")
                


no of topics :: 23
====================New CSV==================
0
====================New CSV==================
1
====================New CSV==================
2
====================New CSV==================
3
====================New CSV==================
4
====================New CSV==================
5
====================New CSV==================
6
====================New CSV==================
7
====================New CSV==================
8
====================New CSV==================
9
====================New CSV==================
10
====================New CSV==================
11
====================New CSV==================
12
====================New CSV==================
13
====================New CSV==================
14
====================New CSV==================
15
====================New CSV==================
16
====================New CSV==================
17
====================New CSV==================
18
====================New CSV==================
19
===========

In [31]:
def starTeller(_value):
    if _value == 0:
        return '*****'
    if _value > 0 and _value <=0.001:
        return '****'
    if _value > 0.001 and _value <=0.01:
        return '***'
    if _value > 0.01 and _value <=0.05:
        return '**'
    if _value > 0.05 and _value <=0.1:
        return '*'
    if _value > 0.1:
        return '.'

In [34]:
noOfTopics = 0
#dfcolumns = ['Manufacturing', 'Transportation', 'Services', 'Finance', 'Retail', 'Others', 'Construction', 'Agriculture']
dfcolumns = ['Sectors','Manufacturing', 'Transportation', 'Services', 'Finance', 'Retail', 'Others', 'Construction', 'Agriculture']

listOfDataframe = [df_Manufacturing, df_Transportation, df_Services, df_Finance, df_Retail, df_Others, df_Construction, df_Agriculture]
#listOfDataframe = [df_Manufacturing, df_Transportation,df_Services]

for name in df_Manufacturing.loc[:,'topic0':'topic22']:
    if 'topic' in name:
        noOfTopics +=1
print("no of topics ::", noOfTopics)

for c in range(noOfTopics):
    print('====================New CSV==================')
    print(c)
    if c not in [12,20,22]:
        topic_num = 'topic' + str(c)
        #print("topic number::", topic_num)
        dft_testScore = pd.DataFrame(columns = dfcolumns)
        count_ = 1
        for j in (listOfDataframe):
            #print("j value ::", j)
            #count_ = 1
            rowStat =[]
            rowStat.append(dfcolumns[count_])
            rowPval =[]
            rowPval.append(dfcolumns[count_])
            rowStarval = []
            rowStarval.append(dfcolumns[count_])
        
            for m in listOfDataframe:
                #print("m value ::", m)
                #print(j[topic_num])
                #if (m == j):   
                arr_df_outerSectore = j[topic_num].to_numpy()
                #print("j head::", j.head(5))
                arr_df_innerSector = m[topic_num].to_numpy()
                # Use scipy.stats.ttest_ind_from_stats.
                t_testScore = stats.ttest_ind(arr_df_outerSectore, arr_df_innerSector)
                Starval = starTeller(t_testScore.pvalue)
               # print("statistic value ::==================",t_testScore.statistic)
                rowStat.append(t_testScore.statistic)
                rowPval.append(t_testScore.pvalue)
                rowStarval.append(Starval)
            count_ = count_ + 1
            a_series = pd.Series(rowStat, index = dft_testScore.columns)
            b_series = pd.Series(rowPval, index = dft_testScore.columns)
            c_series = pd.Series(rowStarval, index = dft_testScore.columns)
            dft_testScore = dft_testScore.append(a_series, ignore_index=True)
            dft_testScore = dft_testScore.append(b_series, ignore_index=True)
            dft_testScore = dft_testScore.append(c_series, ignore_index=True)
        dft_testScore.set_index('Sectors', inplace=True)
        dft_testScore.to_excel("Dataset2_tTest_Star" + str(c) + ".xlsx")
                


no of topics :: 23
====================New CSV==================
0
====================New CSV==================
1
====================New CSV==================
2
====================New CSV==================
3
====================New CSV==================
4
====================New CSV==================
5
====================New CSV==================
6
====================New CSV==================
7
====================New CSV==================
8
====================New CSV==================
9
====================New CSV==================
10
====================New CSV==================
11
====================New CSV==================
12
====================New CSV==================
13
====================New CSV==================
14
====================New CSV==================
15
====================New CSV==================
16
====================New CSV==================
17
====================New CSV==================
18
====================New CSV==================
19
===========

In [6]:
dataset_topic0_ = pd.read_excel('dataset1_t_test_topic0.xlsx',encoding = "utf-8")